In [1]:
from pytorch_lightning import Trainer
import active_dynamicmemory.ActiveDynamicMemoryModel as activedm

import matplotlib.pyplot as plt
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import os
import torchvision.models as models
import torch.nn as nn
import torch
import torch.nn.functional as F
import argparse
import pytorch_lightning as pl
import sklearn 
from sklearn.metrics import confusion_matrix, auc, roc_curve
import torch
import pandas as pd
import pickle
from py_jotools import mut, slurm, cache
import numpy as np
import gc

import hashlib
import dill
import active_dynamicmemory.utils as cutils

from sklearn.metrics import mean_absolute_error
import active_dynamicmemory.runutils as rutils
from run_training import train_config, train_params
import yaml

%load_ext autoreload
%autoreload 

In /home/mperkonigg/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/mperkonigg/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/mperkonigg/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/mperkonigg/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/mperkonigg/.local/lib/python3.6/site-packages/m

## BASE TRAINING

In [ ]:
schedule = True

sparams = {
    'binary': '/home/cir/mperkonigg/env/catinous/bin/python',
    'cwd': '/home/cir/mperkonigg/catinous/active_catinous/',
    'gpu': 1,
    'partition': 'centos7',
    'memory': 100000,
    'constraint': 'bigGPUmem',
    'jobname': 'activecatinous',
    'outputpath': '/home/cir/mperkonigg/slurmoutput/',
    'mailuser': 'matthias.perkonigg@meduniwien.ac.at',
    'minutes': '300'}

hparams={'continuous':False,
         'use_memory':False,
         'datasetfile': '/project/catinous/cardiacslicessplitsingle.csv',
         'noncontinuous_train_splits': ['base'],
         'noncontinuous_steps': 5000,
         'root_dir': '',
         'batch_size': 8,
        'learning_rate': 0.001,
        'droprate': 0.25,
        'val_check_interval': 500,
         'task': 'cardiac',
         'order': ['Siemens', 'GE', 'Philips', 'Canon']
}

droprates = [0.10.25, 0.50]

for dr in droprates:
    hparams['droprate'] = dr

    if schedule:
        slurm.srun(activedm.trained_model, [hparams], params=sparams, remote=True)
    else:
        model, logs, df_cache, basemodel_lr = activedm.trained_model(hparams)

activecatinous08072
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/activecatinous08072.job


# JOINT TRAINING

In [ ]:
run_training

In [7]:
schedule = True

sparams = {
    'binary': '/home/cir/mperkonigg/env/catinous/bin/python',
    'cwd': '/home/cir/mperkonigg/catinous/active_catinous/',
    'gpu': 1,
    'partition': 'centos7',
    'memory': 100000,
    'constraint': 'bigGPUmem',
    'jobname': 'activecatinous',
    'outputpath': '/home/cir/mperkonigg/slurmoutput/',
    'mailuser': 'matthias.perkonigg@meduniwien.ac.at',
    'minutes': '300'}

hparams={'continuous':False,
         'use_memory':False,
         'datasetfile': '/project/catinous/cardiacslicessplitsingle.csv',
         'noncontinuous_train_splits': ['base', 'train'],
         'noncontinuous_steps': 10000,
         'root_dir': '',
         'batch_size': 4,
        'learning_rate': 0.001,
        'droprate': 0.25,
        'val_check_interval': 50,
         'task': 'cardiac',
         'order': ['Siemens', 'GE', 'Philips', 'Canon']
}

for i in range(5):
    hparams['run_postfix'] = i
    hparams['seed'] = i
    if schedule:
        slurm.srun(activedm.trained_model, [hparams], params=sparams, remote=True)
    else:
        model, logs, df_cache, basemodel_lr = activedm.trained_model(hparams)

activecatinous92957
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/activecatinous92957.job
Submitted batch job 3435231

activecatinous28058
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/activecatinous28058.job
Submitted batch job 3435232

activecatinous40972
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/activecatinous40972.job
Submitted batch job 3435233

activecatinous27844
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/activecatinous27844.job
Submitted batch job 3435234

activecatinous24398
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 /scratch/activecatinous24398.job
Submitted batch job 3435235



In [2]:
with open(f'./training_configs/cardiac_joint.yml') as f:
    params = yaml.load(f, Loader=yaml.FullLoader)

for scanner in ['Siemens', 'GE', 'Philips', 'Canon']:
    params['trainparams']['scanner'] = scanner
    
    train_params(params, remote=True, runs=5, jobarray=True)

scheduling job to CIR cluster...
adm43948
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm43948.job
Submitted batch job 3567949

scheduling job to CIR cluster...
adm87984
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm87984.job
Submitted batch job 3567950

scheduling job to CIR cluster...
adm65897
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm65897.job
Submitted batch job 3567951

scheduling job to CIR cluster...
adm45499
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm45499.job
Submitted batch job 3567952

scheduling job to CIR cluster...
adm76074
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm76074.job
Submitted batch job 3567

# Ablation Study - Mem size

In [4]:
#memory size 64, 256, 512, 1024
for lb in [5, 8, 10, 20]:
    with open(f'./training_configs/cardiac_uncertainty_{lb}.yml') as f:
        params = yaml.load(f, Loader=yaml.FullLoader)
    for mem_size in [64, 256, 512, 1024]:
        params['trainparams']['memorymaximum'] = mem_size
        train_params(params, remote=True, runs=5, jobarray=True)

scheduling job to CIR cluster...
adm69007
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm69007.job
Submitted batch job 3567151

scheduling job to CIR cluster...
adm51577
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm51577.job
Submitted batch job 3567152

scheduling job to CIR cluster...
adm38086
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm38086.job
Submitted batch job 3567153

scheduling job to CIR cluster...
adm40726
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm40726.job
Submitted batch job 3567154

scheduling job to CIR cluster...
adm05691
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos jobarray /scratch/adm05691.job
Submitted batch job 3567

In [6]:
with open(f'./training_configs/cardiac_casa_10.yml') as f:
    params = yaml.load(f, Loader=yaml.FullLoader)
    
for s in [6, 7, 8, 9, 10]:
    params['trainparams']['seed'] = s
    params['trainparams']['run_postfix'] = s
    
    train_params(params, remote=True)

scheduling job to CIR cluster...
adm86919
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos normal /scratch/adm86919.job
Submitted batch job 3568629

scheduling job to CIR cluster...
adm38133
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos normal /scratch/adm38133.job
Submitted batch job 3568630

scheduling job to CIR cluster...
adm32794
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos normal /scratch/adm32794.job
Submitted batch job 3568631

scheduling job to CIR cluster...
adm81161
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos normal /scratch/adm81161.job
Submitted batch job 3568632

scheduling job to CIR cluster...
adm30427
sshpass -f ~/.ssh/pass ssh cn1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos normal /scratch/adm30427.job
Submitted batch job 3568633



In [2]:
train_config('./training_configs/ablation_cardiac/cardiac_uncertainty_10_random.yml', remote=True, runs=5)

scheduling job to CIR cluster...
adm61828
sshpass -f ~/.ssh/pass ssh on1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos normal /scratch/adm61828.job
Submitted batch job 3753738

scheduling job to CIR cluster...
adm20767
sshpass -f ~/.ssh/pass ssh on1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos normal /scratch/adm20767.job
Submitted batch job 3753739

scheduling job to CIR cluster...
adm24457
sshpass -f ~/.ssh/pass ssh on1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos normal /scratch/adm24457.job
Submitted batch job 3753740

scheduling job to CIR cluster...
adm44397
sshpass -f ~/.ssh/pass ssh on1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos normal /scratch/adm44397.job
Submitted batch job 3753741

scheduling job to CIR cluster...
adm66599
sshpass -f ~/.ssh/pass ssh on1.cir.meduniwien.ac.at sbatch --export=NONE --partition centos7 --qos normal /scratch/adm66599.job
Submitted batch job 3753742

